In [1]:
import pandas as pd
import string
import re
from transformers import AutoTokenizer, BertForTokenClassification
import numpy as np
from datasets import load_dataset

/home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [62]:
df = pd.read_csv(path_to_train, delimiter="<sep>", index_col="Unnamed: 0", delim_whitespace=True)
df.head()

ValueError: Specified a delimiter with both sep and delim_whitespace=True; you can only specify one.

In [57]:
path_to_train = "../data/preprocessed_data_for_bert_train.csv"
path_to_val = "../data/preprocessed_data_for_bert_val.csv"
data_files = {"train": path_to_train,
             "val": path_to_val}
raw_datasets = load_dataset("csv", data_files=data_files, delimiter='<sep>', index_col="Unnamed: 0",)

Extracting data files: 100%|███████████████████████████████████████| 2/2 [00:00<00:00, 871.09it/s]
Generating train split: 0 examples [00:00, ? examples/s]/home/qklent/programming/machine_learning/alfa_bank_receipts/.venv/lib/python3.11/site-packages/datasets/download/streaming_download_manager.py:804: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)
Failed to read file '/home/qklent/programming/machine_learning/alfa_bank_receipts/data/preprocessed_data_for_bert_train.csv' with error <class 'pandas.errors.ParserError'>: Expected 2 fields in line 2, saw 16. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.


DatasetGenerationError: An error occurred while generating the dataset

In [29]:
raw_datasets["train"][0]["tokens"]

'petmax<sep>бантик<sep>леопард<sep>красн<sep>розой<sep>шт'

In [30]:
label_names = ["O", "B-BRAND", "I-BRAND", "B-GOOD", "I-GOOD"] 

In [31]:
model_checkpoint = "IlyaGusev/rubert_ext_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [32]:
tokenizer.is_fast

True

In [50]:
raw_datasets["train"]["tokens"] = list(map(lambda x: x.split("<sep>"), raw_datasets["train"]["tokens"]))
raw_datasets["train"]["labels"] = list(map(lambda x: x.split("<sep>"), raw_datasets["train"]["labels"]))
raw_datasets["train"]["numeric_labels"] = list(map(lambda x: list(map(int, x.split("<sep>"))), raw_datasets["train"]["numeric_labels"]))


TypeError: 'Dataset' object does not support item assignment

In [49]:
list(map(lambda x: list(map(int, x.split("<sep>"))), raw_datasets["train"]["numeric_labels"]))

[[1, 3, 0, 0, 0, 0],
 [3, 0, 0, 0],
 [3, 1, 2, 0],
 [0, 0, 3, 0, 0, 0, 0],
 [3, 0],
 [1, 3, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0],
 [3, 1, 0, 0],
 [0, 3, 1, 0],
 [3, 0, 0, 0, 0, 1, 0],
 [3, 0],
 [0, 0, 0, 0, 1],
 [3, 0, 0],
 [3, 0, 0, 0, 0, 0],
 [1, 3, 0, 0, 0, 0, 0],
 [3, 0],
 [3, 0, 0, 0, 1],
 [3, 1, 2, 2],
 [0, 3, 1, 2, 0, 0, 0, 0, 0, 0, 0],
 [1, 2, 3, 0, 0, 0, 0, 0],
 [3, 0],
 [3, 0, 0, 1, 2],
 [3, 1, 2, 0],
 [3, 0, 0, 0, 0, 0],
 [0, 3, 1, 0, 0, 0],
 [3, 1, 0, 0, 0, 0, 0],
 [3, 0, 1],
 [3, 0, 0, 1],
 [3, 0, 0],
 [3, 1, 0, 0, 0],
 [0, 0, 1, 3, 0, 0, 0, 0, 0],
 [3, 0, 0, 0],
 [3, 0, 1, 0, 0, 0, 0],
 [1, 0, 3, 0, 0],
 [3, 0, 0, 0],
 [3, 1, 0, 0, 0],
 [0, 3, 0, 0, 0, 0],
 [0, 3, 0, 0],
 [0, 1, 0, 0, 3, 0, 0, 0, 0],
 [3, 0, 0, 0],
 [3, 1, 0, 0, 0, 0],
 [3, 0, 1, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0],
 [3, 1, 2, 0, 0, 0, 0, 0],
 [3, 0, 1, 2],
 [3, 1, 0, 0, 0],
 [3, 0, 0, 0],
 [0, 3, 1, 0],
 [3, 0, 0, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0],
 [1, 2, 0, 0, 0],
 [0, 0, 3, 0, 0, 0, 0,

In [27]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"])
inputs.tokens()

['[CLS]',
 '[',
 "'",
 'pet',
 '##max',
 "'",
 ',',
 "'",
 'бан',
 '##тик',
 "'",
 ',',
 "'",
 'леопард',
 "'",
 ',',
 "'",
 'крас',
 '##н',
 "'",
 ',',
 "'",
 'роз',
 '##ой',
 "'",
 ',',
 "'",
 'шт',
 "'",
 ']',
 '[SEP]']

In [24]:
raw_datasets["train"][0]["tokens"]

"['petmax', 'бантик', 'леопард', 'красн', 'розой', 'шт']"